In [ ]:
 !pip install python-dotenv


In [ ]:
# I hate python warnings
import warnings
warnings.filterwarnings('ignore')

# In order to load environment variables, import these 2 libraries:
import os
from dotenv import load_dotenv

# Load your environment variables
load_dotenv('.env')  # Load environment variables from the file '.env'
API_KEY_FMP = '8fcb0de28daf241dd4522f0f455f2636'
#API_KEY_FMP = os.environ.get('API_KEY_FMP')  # Retrieve the value of the environment variable 'API_KEY_FMP'
#API_KEY_FRED = os.environ.get('API_KEY_FRED')

In [ ]:
# Import the requests library
import requests
import pandas as pd
#def dataextractor(company_tick, start_year, end_year):


# Construct your API request endpoint url
BASE_URL = 'https://financialmodelingprep.com/api/v3'

company_tick = "AAPL"
start_year = 2013
end_year = 2022
#URLS
ratios_url = f"{BASE_URL}/ratios/{company_tick}?period=quarter&apikey={API_KEY_FMP}"
key_metrics_url = f"{BASE_URL}/key-metrics/{company_tick}?period=annual&apikey={API_KEY_FMP}"
balance_sheet_url = f"{BASE_URL}/balance-sheet-statement/{company_tick}?period=annual&apikey={API_KEY_FMP}"
cash_flow_statement_url = f"{BASE_URL}/cash-flow-statement/{company_tick}?period=annual&apikey={API_KEY_FMP}"
stock_dividend_url = f"{BASE_URL}/historical-price-full/stock_dividend/{company_tick}?apikey={API_KEY_FMP}"

#ratios
response = requests.get(ratios_url)
if response.status_code == 429:
  print("FMP API limit reached")
response_dict = response.json()
ratios = pd.DataFrame(response_dict)

#key metrics
response = requests.get(key_metrics_url)
if response.status_code == 429:
  print("FMP API limit reached")
response_dict = response.json()
key_metrics = pd.DataFrame(response_dict)

#balance sheet
response = requests.get(balance_sheet_url)
if response.status_code == 429:
  print("FMP API limit reached")
response_dict = response.json()
balance_sheet = pd.DataFrame(response_dict)

#Dividend Adjusted Return
response = requests.get(stock_dividend_url)
if response.status_code == 429:
  print("FMP API limit reached")
response_dict = response.json()
dividend_paid = pd.DataFrame(response_dict['historical'])

#cash flow statement
response = requests.get(cash_flow_statement_url)
if response.status_code == 429:
  print("FMP API limit reached")
response_dict = response.json()
cash_flow = pd.DataFrame(response_dict)

#print(dataextractor("AAPL", 2013, 2022))

In [ ]:
#NetWorkingCapital
WorkingCapital = key_metrics[["calendarYear", "workingCapital"]]
WorkingCapital.rename(columns={'calendarYear': 'year'}, inplace=True)
WorkingCapital.head()

,year,workingCapital
0,2023,-1742000000
1,2022,-18577000000
2,2021,9355000000
3,2020,38321000000
4,2019,57101000000


In [ ]:
#current liability coverage ratio
CL = cash_flow[["calendarYear","netCashProvidedByOperatingActivities"]]
CL1 = balance_sheet[["calendarYear", "totalCurrentLiabilities"]]
CL = CL.merge(CL1, how="left", on="calendarYear").fillna(0.0)
CL.head()
CL["CurrentLiabilityCoverageRatio"] = CL["netCashProvidedByOperatingActivities"]/CL["totalCurrentLiabilities"]
CL.drop(columns=['netCashProvidedByOperatingActivities','totalCurrentLiabilities'], axis=1, inplace=True)
CL.rename(columns={'calendarYear': 'year'}, inplace=True)
CL['year'] = CL['year'].astype('int64')
CL.head()

,year,CurrentLiabilityCoverageRatio
0,2023,0.760750
1,2022,0.793281
2,2021,0.829114
3,2020,0.765466
4,2019,0.656378


In [ ]:
#Shareholder Equity Ratio
SER = balance_sheet[["calendarYear","totalStockholdersEquity","totalAssets"]]
SER['ShareholderEquityRatio'] = balance_sheet['totalStockholdersEquity']/balance_sheet['totalAssets']
SER.drop(columns=["totalStockholdersEquity", "totalAssets"], axis=1, inplace=True)
SER.rename(columns={'calendarYear': 'year'}, inplace=True)
SER.head()

,year,ShareholderEquityRatio
0,2023,0.176259
1,2022,0.143646
2,2021,0.179743
3,2020,0.201733
4,2019,0.267308


In [ ]:
#Debts-to-Assets Ratio
DAR = balance_sheet[["calendarYear","totalDebt","totalAssets"]]
DAR['Debts-to-Assets Ratio'] = balance_sheet['totalDebt']/balance_sheet['totalAssets']
DAR.drop(columns=["totalDebt","totalAssets"], axis=1,inplace=True)
DAR.rename(columns={'calendarYear': 'year'}, inplace=True)
DAR.head()

,year,Debts-to-Assets Ratio
0,2023,0.347025
1,2022,0.340375
2,2021,0.355323
3,2020,0.347145
4,2019,0.319178


In [ ]:
#Cash Flow Coverage Ratio = Net Operating Cash Flow/ Total Debt
cashflow = cash_flow[["calendarYear","operatingCashFlow"]]
debt = balance_sheet[["calendarYear","totalDebt"]]
CFCR = cashflow.merge(debt, on='calendarYear', how='left').fillna(0.0)
CFCR.rename(columns={'calendarYear': 'year'}, inplace=True)
CFCR['CashFlowCoverageRatio'] = cash_flow["operatingCashFlow"]/balance_sheet["totalDebt"]
CFCR.drop(columns=["operatingCashFlow","totalDebt"], axis=1, inplace=True)
CFCR.head()

,year,CashFlowCoverageRatio
0,2023,0.903461
1,2022,1.017340
2,2021,0.834179
3,2020,0.717510
4,2019,0.642230


In [ ]:
dividends=dividend_paid[["date","adjDividend"]]
start_year = 2013
end_year = 2022

end_year = end_year + 1

list(range(start_year, end_year))

# Data Transformation
if dividends.shape == (0, 0):  # Handle the case where the company never issued any dividend in the past
    dividends = pd.DataFrame({
        "year": list(range(start_year - 1, end_year + 1)),
        "adjDividend": [0.0] * len(list(range(start_year - 1, end_year + 1)))  # We are obtaining 2 more years' data
    })
else:
    # Extract year data from the date column
    dividends['year'] = pd.to_datetime(dividends['date']).dt.year
    dividends = dividends.groupby("year").agg({"adjDividend": "sum"}).reset_index()
    all_years = pd.DataFrame({'year': list(range(start_year - 1, end_year + 1))})
    dividends = all_years.merge(dividends, on='year', how='left').fillna(0.0)
dividends

,year,adjDividend
0,2012,0.189286
1,2013,0.421433
2,2014,0.461430
3,2015,0.507500
4,2016,0.557500
5,2017,0.615000
6,2018,0.705000
7,2019,0.760000
8,2020,0.807500
9,2021,0.865000


In [ ]:
#all ratios merged into a single table
new = CFCR.merge(SER, how='left', on='year').fillna(0.0)
new = new.merge(WorkingCapital, how='left', on='year').fillna(0.0)
new = new.merge(DAR, how='left', on='year').fillna(0.0)
new['year'] = new['year'].astype('int64')
new = new.merge(dividends, how='right', on='year').fillna(0.0)
new = new.merge(CL, how='left',on='year').fillna(0.0)
new

,year,CashFlowCoverageRatio,ShareholderEquityRatio,workingCapital,Debts-to-Assets Ratio,adjDividend,CurrentLiabilityCoverageRatio
0,2012,inf,0.671404,19111000000,0.000000,0.189286,1.319496
1,2013,3.164269,0.596855,29628000000,0.081932,0.421433,1.229236
2,2014,1.691826,0.481140,5083000000,0.152239,0.461430,0.941133
3,2015,1.260681,0.410890,8768000000,0.221916,0.507500,1.008138
4,2016,0.756320,0.398678,27863000000,0.270550,0.557500,0.833152
5,2017,0.549775,0.357155,27831000000,0.308218,0.615000,0.630845
6,2018,0.676380,0.292971,14473000000,0.313030,0.705000,0.662588
7,2019,0.642230,0.267308,57101000000,0.319178,0.760000,0.656378
8,2020,0.717510,0.201733,38321000000,0.347145,0.807500,0.765466
9,2021,0.834179,0.179743,9355000000,0.355323,0.865000,0.829114


In [ ]:
metrics = ratios[['symbol','date', 'cashPerShare', 'freeCashFlowPerShare', 'operatingCashFlowPerShare', 'netProfitMargin', 'grossProfitMargin','debtEquityRatio',
                'longTermDebtToCapitalization', 'totalDebtToCapitalization', 'cashRatio', 'dividendYield', 'dividendPayoutRatio', 'payoutRatio', 'daysOfSalesOutstanding',
                'daysOfInventoryOutstanding', 'operatingCycle', 'daysOfPayablesOutstanding', 'cashConversionCycle', 'returnOnAssets', 'returnOnEquity', 'returnOnCapitalEmployed',
                'priceBookValueRatio', 'priceToBookRatio', 'priceToSalesRatio']]

In [ ]:
#used the quarterly information and used the data of 2020 from 2020-12-31 and similarly for all years
metrics = metrics[pd.to_datetime(metrics['date']).dt.month==12]

In [ ]:
temporary = metrics
metrics.head(5)


,symbol,date,cashPerShare,freeCashFlowPerShare,operatingCashFlowPerShare,netProfitMargin,grossProfitMargin,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,...,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,priceBookValueRatio,priceToBookRatio,priceToSalesRatio
1,AAPL,2023-12-30,4.713160,2.418025,2.572251,0.283638,0.458750,1.458030,0.562026,0.593170,...,9.054234,46.764290,80.858158,-34.093868,0.095940,0.457706,0.183897,40.298174,40.298174,24.972567
5,AAPL,2022-12-31,3.231353,1.901373,2.139659,0.256056,0.429623,1.958679,0.637189,0.662011,...,9.185598,50.807736,78.007542,-27.199806,0.086513,0.528813,0.171946,36.401387,36.401387,17.625873
9,AAPL,2021-12-31,3.899102,2.694225,2.865226,0.279398,0.437638,1.707140,0.597157,0.630606,...,7.587157,54.969221,96.017044,-41.047823,0.090847,0.481427,0.177590,40.464306,40.464306,23.483629
13,AAPL,2020-12-31,4.536490,2.082241,2.288912,0.258034,0.397778,1.691879,0.599867,0.628512,...,6.669100,54.011593,85.621433,-31.609840,0.081216,0.434208,0.151363,33.932123,33.932123,20.164583
17,AAPL,2019-12-31,6.068008,1.608649,1.727957,0.242172,0.383548,1.209548,0.509712,0.547419,...,6.514434,45.669075,71.728738,-26.059662,0.065281,0.248361,0.107227,14.480262,14.480262,14.119434


In [ ]:
# Data Transformation
if temporary.shape == (0, 0):  # Handle the case where there is a company that never issued any dividend in the past and the table is empty; thus it will create a new table.
    dividends = pd.DataFrame({     #create a new dataframe
        "year": list(range(start_year - 1, end_year + 1)),
        "ratios": [0.0] * len(list(range(start_year - 1, end_year + 1)))  # We are obtaining 2 more years' data
    })
else:
    temporary['year'] = pd.to_datetime(temporary['date']).dt.year           #why is "dt" in the code necessary?
    all_years = pd.DataFrame({'year': list(range(start_year - 1, end_year + 1))})
    temporary = all_years.merge(temporary, on='year', how='left').fillna(0.0)

temporary.drop(['date'],axis=1,inplace=True)

In [ ]:
temporary = temporary.merge(new, how='left', on='year')
temporary

,year,symbol,cashPerShare,freeCashFlowPerShare,operatingCashFlowPerShare,netProfitMargin,grossProfitMargin,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,...,returnOnCapitalEmployed,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,CashFlowCoverageRatio,ShareholderEquityRatio,workingCapital,Debts-to-Assets Ratio,adjDividend,CurrentLiabilityCoverageRatio
0,2012,AAPL,1.514665,0.797690,0.891073,0.239910,0.386337,0.000000,0.000000,0.000000,...,0.115342,3.924475,3.924475,9.168004,inf,0.671404,19111000000,0.000000,0.189286,1.319496
1,2013,AAPL,1.622598,0.822080,0.903547,0.226968,0.379310,0.130787,0.115660,0.115660,...,0.101876,3.877147,3.877147,8.730144,3.164269,0.596855,29628000000,0.081932,0.421433,1.229236
2,2014,AAPL,1.388950,1.303122,1.442817,0.241612,0.398678,0.295172,0.208584,0.227902,...,0.128774,5.228679,5.228679,8.644118,1.691826,0.481140,5083000000,0.152239,0.461430,0.941133
3,2015,AAPL,1.712290,1.054924,1.235085,0.242000,0.400978,0.490875,0.293182,0.329253,...,0.111289,4.562702,4.562702,7.713572,1.260681,0.410890,8768000000,0.221916,0.507500,1.008138
4,2016,AAPL,2.852230,1.115187,1.276549,0.228344,0.385139,0.661296,0.357165,0.398060,...,0.094567,4.636278,4.636278,7.833938,0.756320,0.398678,27863000000,0.270550,0.557500,0.833152
5,2017,AAPL,3.772485,1.238491,1.383419,0.227255,0.384085,0.873045,0.425699,0.466110,...,0.090287,6.171965,6.171965,9.800361,0.549775,0.357155,27831000000,0.308218,0.615000,0.630845
6,2018,AAPL,4.562409,1.231835,1.408943,0.236805,0.379919,0.973179,0.440955,0.493204,...,0.087953,6.337351,6.337351,8.861617,0.676380,0.292971,14473000000,0.313030,0.705000,0.662588
7,2019,AAPL,6.068008,1.608649,1.727957,0.242172,0.383548,1.209548,0.509712,0.547419,...,0.107227,14.480262,14.480262,14.119434,0.642230,0.267308,57101000000,0.319178,0.760000,0.656378
8,2020,AAPL,4.536490,2.082241,2.288912,0.258034,0.397778,1.691879,0.599867,0.628512,...,0.151363,33.932123,33.932123,20.164583,0.717510,0.201733,38321000000,0.347145,0.807500,0.765466
9,2021,AAPL,3.899102,2.694225,2.865226,0.279398,0.437638,1.707140,0.597157,0.630606,...,0.177590,40.464306,40.464306,23.483629,0.834179,0.179743,9355000000,0.355323,0.865000,0.829114


In [ ]:
list(temporary.columns) #contains a cleaner version of the ratio table with our predictors and all years.

['year',
 'symbol',
 'cashPerShare',
 'freeCashFlowPerShare',
 'operatingCashFlowPerShare',
 'netProfitMargin',
 'grossProfitMargin',
 'debtEquityRatio',
 'longTermDebtToCapitalization',
 'totalDebtToCapitalization',
 'cashRatio',
 'dividendYield',
 'dividendPayoutRatio',
 'payoutRatio',
 'daysOfSalesOutstanding',
 'daysOfInventoryOutstanding',
 'operatingCycle',
 'daysOfPayablesOutstanding',
 'cashConversionCycle',
 'returnOnAssets',
 'returnOnEquity',
 'returnOnCapitalEmployed',
 'priceBookValueRatio',
 'priceToBookRatio',
 'priceToSalesRatio',
 'CashFlowCoverageRatio',
 'ShareholderEquityRatio',
 'workingCapital',
 'Debts-to-Assets Ratio',
 'adjDividend',
 'CurrentLiabilityCoverageRatio']

In [ ]:
# Let's create our target variable
import numpy as np
columns = list(temporary)[2:]
for i in columns:   #through columns 3 to 26
  temporary[f"next_year_{i}"] = temporary[i].shift(-1)   #shift of -1 causes the new column to be shifted one unit up!

  conditions = [
      temporary[i] <= temporary[f"next_year_{i}"],
      temporary[i] > temporary[f"next_year_{i}"]
  ]

  choices = ['constant/increased', 'decreased']

  # Create the target column 'dps_change' based on the conditions
  temporary[f"{i}_change_next_year"] = np.select(conditions, choices, default=np.nan)    #this replaces the need for an if/else statement.

In [ ]:
temporary
adjDiv_nextyear = temporary[["adjDividend_change_next_year"]]
adjDiv_nextyear

,adjDividend_change_next_year
0,constant/increased
1,constant/increased
2,constant/increased
3,constant/increased
4,constant/increased
5,constant/increased
6,constant/increased
7,constant/increased
8,constant/increased
9,constant/increased


In [ ]:
columns = list(temporary)[2:31]   #if adding a new metric, increase the end limit of this
columns

['cashPerShare',
 'freeCashFlowPerShare',
 'operatingCashFlowPerShare',
 'netProfitMargin',
 'grossProfitMargin',
 'debtEquityRatio',
 'longTermDebtToCapitalization',
 'totalDebtToCapitalization',
 'cashRatio',
 'dividendYield',
 'dividendPayoutRatio',
 'payoutRatio',
 'daysOfSalesOutstanding',
 'daysOfInventoryOutstanding',
 'operatingCycle',
 'daysOfPayablesOutstanding',
 'cashConversionCycle',
 'returnOnAssets',
 'returnOnEquity',
 'returnOnCapitalEmployed',
 'priceBookValueRatio',
 'priceToBookRatio',
 'priceToSalesRatio',
 'CashFlowCoverageRatio',
 'ShareholderEquityRatio',
 'workingCapital',
 'Debts-to-Assets Ratio',
 'adjDividend',
 'CurrentLiabilityCoverageRatio']

In [ ]:
# We can also create a predictor here - dps change from last year
for i in columns:
  temporary[f"last_year_{i}"] = temporary[i].shift(1)    #creates a new column where the column is shifted one unit down from original data.
  temporary[f"{i}_growth"] = temporary[i] - temporary[f"last_year_{i}"]

In [ ]:
# Another predictor that we can create is dividend change as a percentage, called dps_growth_rate
for i in columns:
  temporary[f"{i}_growth_rate"] = np.where(
      (temporary[f"last_year_{i}"] == 0) & (temporary[i] == 0),
      0,  # If both are 0 then change is 0
      np.where(
          temporary[f"last_year_{i}"] != 0,
          ((temporary[i] / temporary[f"last_year_{i}"]) - 1) * 100, 999  # If last year dividend is 0 then return 999
      )
  )

final = pd.DataFrame(temporary[["year","symbol"]])

# Only keep the columns that we need
for i in columns:
  final[i] = temporary[[i]]
  final[f"{i}_growth"] = temporary[[f"{i}_growth"]]
  final[f"{i}_growth_rate"] = temporary[[f"{i}_growth_rate"]]

In [ ]:
"""
summary of data collection and organization process:
1. created an adj column with years
2. created next year dividend
3. created dps_change_next_year
4. created last year dividend column
5. created dps_growth column
6. created dps_growth_rate column

"""

'\nsummary of data collection and organization process:\n1. created an adj column with years\n2. created next year dividend\n3. created dps_change_next_year\n4. created last year dividend column\n5. created dps_growth column\n6. created dps_growth_rate column\n\n'

In [ ]:
list(final.columns)

['year',
 'symbol',
 'cashPerShare',
 'cashPerShare_growth',
 'cashPerShare_growth_rate',
 'freeCashFlowPerShare',
 'freeCashFlowPerShare_growth',
 'freeCashFlowPerShare_growth_rate',
 'operatingCashFlowPerShare',
 'operatingCashFlowPerShare_growth',
 'operatingCashFlowPerShare_growth_rate',
 'netProfitMargin',
 'netProfitMargin_growth',
 'netProfitMargin_growth_rate',
 'grossProfitMargin',
 'grossProfitMargin_growth',
 'grossProfitMargin_growth_rate',
 'debtEquityRatio',
 'debtEquityRatio_growth',
 'debtEquityRatio_growth_rate',
 'longTermDebtToCapitalization',
 'longTermDebtToCapitalization_growth',
 'longTermDebtToCapitalization_growth_rate',
 'totalDebtToCapitalization',
 'totalDebtToCapitalization_growth',
 'totalDebtToCapitalization_growth_rate',
 'cashRatio',
 'cashRatio_growth',
 'cashRatio_growth_rate',
 'dividendYield',
 'dividendYield_growth',
 'dividendYield_growth_rate',
 'dividendPayoutRatio',
 'dividendPayoutRatio_growth',
 'dividendPayoutRatio_growth_rate',
 'payoutRati

In [ ]:
final

,year,symbol,cashPerShare,cashPerShare_growth,cashPerShare_growth_rate,freeCashFlowPerShare,freeCashFlowPerShare_growth,freeCashFlowPerShare_growth_rate,operatingCashFlowPerShare,operatingCashFlowPerShare_growth,...,workingCapital_growth_rate,Debts-to-Assets Ratio,Debts-to-Assets Ratio_growth,Debts-to-Assets Ratio_growth_rate,adjDividend,adjDividend_growth,adjDividend_growth_rate,CurrentLiabilityCoverageRatio,CurrentLiabilityCoverageRatio_growth,CurrentLiabilityCoverageRatio_growth_rate
0,2012,AAPL,1.514665,NaN,NaN,0.797690,NaN,NaN,0.891073,NaN,...,NaN,0.000000,NaN,NaN,0.189286,NaN,NaN,1.319496,NaN,NaN
1,2013,AAPL,1.622598,0.107933,7.125866,0.822080,0.024390,3.057524,0.903547,0.012473,...,55.031134,0.081932,0.081932,999.000000,0.421433,0.232147,122.643951,1.229236,-0.090259,-6.840438
2,2014,AAPL,1.388950,-0.233647,-14.399590,1.303122,0.481042,58.515259,1.442817,0.539271,...,-82.843931,0.152239,0.070307,85.810901,0.461430,0.039997,9.490776,0.941133,-0.288103,-23.437596
3,2015,AAPL,1.712290,0.323340,23.279433,1.054924,-0.248198,-19.046409,1.235085,-0.207733,...,72.496557,0.221916,0.069677,45.768050,0.507500,0.046070,9.984180,1.008138,0.067005,7.119616
4,2016,AAPL,2.852230,1.139940,66.574009,1.115187,0.060263,5.712558,1.276549,0.041464,...,217.780566,0.270550,0.048633,21.915176,0.557500,0.050000,9.852217,0.833152,-0.174986,-17.357350
5,2017,AAPL,3.772485,0.920255,32.264390,1.238491,0.123303,11.056723,1.383419,0.106870,...,-0.114848,0.308218,0.037668,13.922872,0.615000,0.057500,10.313901,0.630845,-0.202307,-24.282125
6,2018,AAPL,4.562409,0.789925,20.939113,1.231835,-0.006655,-0.537375,1.408943,0.025524,...,-47.996838,0.313030,0.004812,1.561387,0.705000,0.090000,14.634146,0.662588,0.031743,5.031823
7,2019,AAPL,6.068008,1.505599,33.000084,1.608649,0.376814,30.589628,1.727957,0.319014,...,294.534651,0.319178,0.006148,1.964069,0.760000,0.055000,7.801418,0.656378,-0.006210,-0.937179
8,2020,AAPL,4.536490,-1.531518,-25.239225,2.082241,0.473592,29.440354,2.288912,0.560955,...,-32.889091,0.347145,0.027966,8.761958,0.807500,0.047500,6.250000,0.765466,0.109088,16.619651
9,2021,AAPL,3.899102,-0.637388,-14.050253,2.694225,0.611984,29.390661,2.865226,0.576314,...,-75.587798,0.355323,0.008178,2.355810,0.865000,0.057500,7.120743,0.829114,0.063648,8.314869


In [ ]:
final = final.loc[(temporary['year'] >= start_year) & (temporary['year'] <= end_year - 1)]
final["adjDiv_change_next_year"] = adjDiv_nextyear
final

NameError: name 'final' is not defined

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
nan_count = final.isna().sum()

#print(nan_count)

In [ ]:
final.isnull().any(axis=1).sum()

0

In [ ]:
#next steps
#1. need to concatenate the table for all the companies using a python functions

In [ ]:
from pandas import DataFrame,Series
from sklearn.decomposition import PCA
import  matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler